# Interactive Lyot coronagraph

M Kenworthy // Leiden Observatory // kenworthy@strw.leidenuniv.nl

Based on E. Por hcipy tutorials


The next cell is used in Google Colab to install `hcipy` - skip it if you already have it installed

In [1]:
%pip install hcipy

Note: you may need to restart the kernel to use updated packages.


In [5]:
from hcipy import *
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches


import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import FloatSlider



In [8]:
pupil_grid = make_pupil_grid(128, 1.5)
focal_grid = make_focal_grid(8, 12)
prop = FraunhoferPropagator(pupil_grid, focal_grid)

aperture = evaluate_supersampled(make_circular_aperture(1), pupil_grid, 4)


def placid(xc,yc,xs,ys,fig):
    'placid - given centre (xc,yc) and side lengths (xs,ys) and Figure return 4-tuple suitable for Figure.add_axes'

    w = fig.get_figwidth()
    h = fig.get_figheight()
    R = w/h

    d = lambda a,R: a/R+0.5 # correctly adjusts the aspect ratio for the Figure

    # calculate lower left and upper right corners of the Axes
    x1=d(xc-(xs/2),R)
    x2=d(xc+(xs/2),R)
    y1=d(yc-(ys/2),1.)
    y2=d(yc+(ys/2),1.)

    # return the lower left corner and the width and height
    return ((x1,y1,x2-x1,y2-y1))


def s(d):
	'FREE SPACE: d - distance'
	return np.array([[1 , d],
    	[0, 1]])

def f(f):
	'LENS: f - focal length'
	return np.array([[1 , 0],
    	[-1/f, 1]])


def make_coro_rays(fp1, dx=5):

	to_lens1 = np.matmul(s(dx),fp1)
	lens1 = np.matmul(f(dx),to_lens1)
	to_pp1 = np.matmul(s(dx),lens1)
	to_lens2 = np.matmul(s(dx),to_pp1)
	lens2 = np.matmul(f(dx),to_lens2)
	to_fp2 = np.matmul(s(dx),lens2)

	rays = np.stack((fp1,to_lens1,to_pp1,to_lens2,to_fp2))

	# pull out the y positions of the rays at each optical element
	ray = rays[:,0,:]

	return ray


def addtext(a,s):
    a.text(0.05, 0.95, s, ha='left', va='top', transform=a.transAxes, color='white', fontsize=20)

def ff(dy, mdim, b):
    
    figsize = (14,6)
    fig = plt.figure(figsize=figsize) #, facecolor='lightskyblue')

    # this is the whole frame, including the images
    a1 = fig.add_axes([0.1,0.1,0.8,0.8])

    # an inserted figure that contains the ray trace and optical lenses
    axray = fig.add_axes([0.15,0.2,0.7,0.4])

    # set up the individual images in the coronagraph chain
    a1.set_xlim(0,1)
    a1.set_ylim(0,1)

    # parameters for spacing the images
    s = 0.2
    p = 0.15
    y = 0.28

    # set up the images along the top of the figure
    x2 = fig.add_axes(placid(-5*p, y, s, s, fig))
    x3 = fig.add_axes(placid(-3*p, y, s, s, fig))
    x4 = fig.add_axes(placid(  -p, y, s, s, fig))
    x5 = fig.add_axes(placid(   p, y, s, s, fig))
#    x6 = fig.add_axes(placid( 3*p, y, s, s, fig))
    x6 = fig.add_axes(placid( 5*p, y-s/2, s*1.8, s*1.8, fig))

    for a in (a1,axray,x2,x3,x4,x5,x6):
        a.axis('off')
    
    wf = Wavefront(aperture * np.exp(2j * np.pi * pupil_grid.y * dy))

    img_ref = prop(wf).intensity
    
    focal_plane_mask_diameter = mdim
    focal_plane_mask_offset = [0, 0]

    focal_plane_mask_grid = make_focal_grid(64, focal_plane_mask_diameter / 2).shifted(focal_plane_mask_offset)

    focal_plane_mask = lambda grid: 1 - make_circular_aperture(focal_plane_mask_diameter, center=focal_plane_mask_offset)(grid)

    focal_plane_mask_evaluated = evaluate_supersampled(focal_plane_mask, focal_plane_mask_grid, 8)

    lyot_stop = evaluate_supersampled(make_circular_aperture(b), pupil_grid, 4)

    
    # make two coronagraph propagations to pupil plane - one with lyot mask, one without
    coro = LyotCoronagraph(pupil_grid, focal_plane_mask_evaluated, lyot_stop)
    coro_without_lyot = LyotCoronagraph(pupil_grid, focal_plane_mask_evaluated, None)

    img_before_fpm = prop(wf)

    imshow_field(np.log10(img_before_fpm.intensity / img_ref.max()), vmin=-5, vmax=0, cmap='inferno', ax=x2)
    addtext(x2,"B")

    img_after_fpm = img_before_fpm.copy()
    img_after_fpm.electric_field *=focal_plane_mask(img_after_fpm.electric_field.grid)

    C = img_after_fpm.intensity
    C[(C<1e-5)] = 1e-5
    
#    print(C.min())
    
    imshow_field(np.log10(C / img_ref.max()), vmin=-5, vmax=0, cmap='inferno', ax=x3)
    addtext(x3,"C")

    img_before_lyot_stop = coro_without_lyot(wf)

    imshow_field(img_before_lyot_stop.intensity, vmax=1, cmap='inferno', ax=x4)
    addtext(x4,"D")

    img_after_lyot_stop = coro(wf)
    
    imshow_field(img_after_lyot_stop.intensity, vmax=1, cmap='inferno', ax=x5)
    addtext(x5,"E")

    img_science_plane = prop(coro(wf))
    imshow_field(np.log10(img_science_plane.intensity/img_ref.max()), vmin=-5, vmax=0, cmap='inferno', ax=x6)
    addtext(x6,"F")

    x2.scatter(0,0,marker='.',color='white')
    x5.add_patch(patches.Circle((0,0), radius=0.5, fill=None,edgecolor='white',linestyle='--'))

    ratio_lens_fp = 30.

    # fan of rays from a single point
    thetas = np.linspace(-0.1,0.1,17)
    x = np.zeros_like(thetas)+dy/ratio_lens_fp
    fp1 = np.vstack((x,thetas))
    
    # draw all the rays through the coronagraph
    ray = make_coro_rays(fp1)
    
    xpos = np.arange(ray.shape[0])
        
    axray.plot(xpos[0:3],ray[0:3], color='blue', zorder=5)
    
    lens1 = mpl.patches.Ellipse((1.0, 0.0), 0.05, 1.4, color='gray',alpha=0.8)
    lens2 = mpl.patches.Ellipse((3.0, 0.0), 0.05, 1.4, color='gray',alpha=0.8)
    
    # TODO add color blind friendly pallette

    # add the lenses
    axray.add_artist(lens1)
    axray.add_artist(lens2)
    axray.set_ylim(-0.75,0.75)
    
    pupmax = b/2
    
    # add the Lyot stop
    lyot1 = mpl.patches.Rectangle((2.0,pupmax),0.05,(0.7-pupmax),color='blue')
    lyot2 = mpl.patches.Rectangle((2.0,-pupmax),0.05,-(0.7-pupmax),color='blue')
    axray.add_artist(lyot1)
    axray.add_artist(lyot2)

    dy_mask = mdim/ratio_lens_fp/2
    fpm1 = mpl.patches.Rectangle((-0.02,-dy_mask),0.02,(2*dy_mask),color='red')
    axray.add_artist(fpm1)

    x2.annotate('', xy=(-10,-10), xytext=(-0.05,0), xycoords=x2.transData, 
         textcoords=axray.transData, 
         arrowprops=dict(facecolor='black', arrowstyle='<-', clip_on=False))

    x3.annotate('', xy=(-10,-10), xytext=(0,0), xycoords=x3.transData, 
         textcoords=axray.transData, 
         arrowprops=dict(facecolor='black', arrowstyle='<-', clip_on=False))
        
    x4.annotate('', xy=(0,-0.5), xytext=(2.0,0.6), xycoords=x4.transData, 
         textcoords=axray.transData, 
         arrowprops=dict(facecolor='black', arrowstyle='<-', clip_on=False))
    
    x5.annotate('', xy=(0,-0.5), xytext=(2.05,0.6), xycoords=x5.transData, 
         textcoords=axray.transData, 
         arrowprops=dict(facecolor='black', arrowstyle='<-', clip_on=False))
    
    x6.annotate('', xy=(10,-10), xytext=(4.01,0.0), xycoords=x6.transData, 
         textcoords=axray.transData, 
         arrowprops=dict(facecolor='black', arrowstyle='<-', clip_on=False))

    axray.hlines(0,-1,5, linestyle='dotted')
    
    clipped = (np.abs(ray[2])<=pupmax)
    
    t = ray[2:]
    
    axray.plot(xpos[2:],t[:,clipped], color='blue', alpha=0.5)

    plt.show()

#interactive_plot = interactive(ff, dy=(0.0, 5.0), mdim=(0.5,4.0), b=(0.2, 1.5))

interactive_plot = interactive(ff, dy=FloatSlider(description='Source offset',
                                                 min=0.0, max=5.0,step=0.1, value=1.2,
                                                 style=dict(description_width='120px')), 
                               mdim=FloatSlider(description='Mask diameter',
                                                min=0.5,max=4.0,step=0.1, value=3,
                                               style=dict(description_width='120px')), 
                               b=FloatSlider(description='Lyot Stop diameter',
                                             min=0.2, max=1.2,step=0.1, value=0.8,
                                            style=dict(description_width='120px'))
                              )
output = interactive_plot.children[-1]
output.layout.height = '600px'
interactive_plot

interactive(children=(FloatSlider(value=1.2, description='Source offset', max=5.0, style=SliderStyle(descripti…